In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
taste = pd.read_csv('./data/train_triplets.txt', sep='\t', header=None, names=['user_id','song_id','play_count'])

In [3]:
taste.shape

(48373586, 3)

In [4]:
taste.head()

,user_id,song_id,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [5]:
unique_user = taste.user_id.unique()
unique_song = taste.song_id.unique()

In [6]:
len(unique_user), len(unique_song)

(1019318, 384546)

In [7]:
#grouped_taste = taste.groupby(['user_id', 'song_id']).sum().reset_index() 

In [8]:
#grouped_taste.shape

#### Construct sparse matrix: users-songs

In [9]:
users = list(np.sort(unique_user)) # Get our unique users
songs = list(unique_song) # Get our unique songs
playcount = list(taste.play_count) # All of our playcount

In [10]:
rows = taste.user_id.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = taste.song_id.astype('category', categories = songs).cat.codes 
# Get the associated column indices
playcount_sparse = sparse.csr_matrix((playcount, (rows, cols)), shape=(len(users), len(songs)))

In [11]:
playcount_sparse

<1019318x384546 sparse matrix of type '<type 'numpy.int64'>'
	with 48373586 stored elements in Compressed Sparse Row format>

#### Mask sparse matrix to create training dataset

In [12]:
import random

In [25]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    #nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    num_samples = int(np.ceil(pct_test*len(nonzero_inds[0]))) # Round the number of samples needed to the nearest integer
    random.seed(0) # Set the random seed to zero for reproducibility
    samples_0 = random.sample(nonzero_inds[0], num_samples) # Sample a random number of user-item pairs without replacement
    random.seed(1)
    samples_1 = random.sample(nonzero_inds[1], num_samples)
    user_inds = samples_0 # Get the user row indices
    item_inds = samples_1 # Get the item column indices
    #user_inds = [index[0] for index in samples] # Get the user row indices
    #item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [26]:
X_train, X_test, X_users_altered = make_train(playcount_sparse, pct_test = 0.2)

/usr/local/lib/python2.7/dist-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [28]:
X_train.shape, X_test.shape, len(X_users_altered)

((1019318, 384546), (1019318, 384546), 998678)

In [29]:
X_train[X_train != 0].shape, X_test[X_test != 0].shape

((1, 48281656), (1, 48373586))

In [31]:
import implicit

In [32]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((X_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                          iterations = 50)

In [33]:
user_vecs.shape, item_vecs.shape

((1019318, 20), (384546, 20))

In [39]:
np.savetxt("./data/users_vecs_rank20.csv", user_vecs, delimiter=",")
np.savetxt("./data/items_vecs_rank20.csv", item_vecs, delimiter=",")

#### Evaluation

In [40]:
from sklearn import metrics

In [41]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [43]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        print pred[0], actual[0]
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [ ]:
calc_mean_auc(X_train, X_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], X_test)

0.0286891803855 0
0.0109046558323 0
0.00292943090533 0
0.0395812744163 0
-0.00239219656451 0
-0.0325885559384 0
0.0819153634121 0
-0.0356772526837 0
-0.043023728494 0
0.00106188302762 0
0.0309226124109 0
-0.0175747505939 0
0.0880413756272 0
0.0673183464115 0
-0.058278388388 0
0.21569335774 0
0.00414019911136 0
0.596915785815 0
-0.0548752586393 0
0.00814063776533 0
-0.00374241563011 0
0.0230912208953 0
0.0218756230065 0
0.0961294796425 0
0.142391224027 0
0.106982555925 0
0.252275100179 0
-0.0273379136009 0
-0.0373791654441 0
-0.0218937590652 0
0.405118973558 0
-0.0310236894938 0
0.169772511107 0
0.0346528180512 0
0.121729089002 0
0.0588743522743 0
0.422884805773 0
0.0177003107429 0
0.00223240316034 0
0.185569537325 0
-0.0223403305157 0
-0.0542833998998 0
0.0503892036619 0
-0.0488522024043 0
0.103683013058 0
-0.0235692737771 0
0.302037602749 0
0.0051459956609 0
-0.0841728948331 0
-0.0568821740138 0
0.00219400884629 0
0.229062625393 0
-0.0403294897527 0
-0.0264296613096 0
0.143982030639 0